<a href="https://colab.research.google.com/github/ecloguehwang/HSS/blob/master/%EC%9A%B0%EC%9E%A5%EC%82%B0%ED%9E%90%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%8A%B8%EC%95%84%ED%8C%8C%ED%8A%B8_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#colab에서 matplotlib와 sns 라이브러리 그래프 한글을 깨지지 않게 하는 법A: 이 코드실행하고 런타임(runtime) 다시 실행하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [27]:
#드디어 수정 성공!: Deepseek와 chatGPT의  합작품

!pip install selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

def get_data():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    # URL 설정
    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL'
    driver.get(base_url)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 아파트 항목 컨테이너 선택 (실제 페이지 구조에 맞게 조정 필요)
    containers = soup.find_all('div', class_='item_inner')  # 예시 클래스명

    apartment_info = []
    transaction_types = []
    prices = []
    property_specs = []
    real_estate_offices = []

    for container in containers:
        # 동호수 정보
        info_span = container.find('span', class_='text')
        apartment_info.append(info_span.get_text(strip=True) if info_span else '')

        # 거래 방식
        type_span = container.find('span', class_='type')
        transaction_types.append(type_span.get_text(strip=True) if type_span else '')

        # 가격 정보
        price_span = container.find('span', class_='price')
        prices.append(price_span.get_text(strip=True) if price_span else '')

        # 물건정보 (1개 또는 2개를 '/'로 구분하여 결합)
        specs = container.find_all('span', class_='spec')
        specs_text = [spec.get_text(strip=True) for spec in specs]
        property_specs.append(' / '.join(specs_text))


        # 부동산 정보 (BeautifulSoup로 직접 추출)
        agent_anchor = container.find('a', class_='agent_name', attrs={'data-nclk': lambda x: x and 'TAA.realtor' in x})
        real_estate_offices.append(agent_anchor.get_text(strip=True) if agent_anchor else '')



    driver.quit()

    return apartment_info, transaction_types, prices, property_specs, real_estate_offices

# 데이터 수집 실행
apartment_info, transaction_types, prices, property_specs, real_estate_offices = get_data()

# 수집된 데이터 출력
print("동호수:", apartment_info)
print("거래 방식:", transaction_types)
print("가격:", prices)
print("물건설명:", property_specs)
print("부동산:", real_estate_offices)

# 데이터프레임으로 저장
data_list = list(zip(apartment_info, transaction_types, prices, property_specs, real_estate_offices))
df = pd.DataFrame(data_list, columns=['동호수', '거래 방식', '가격', '물건 설명', '부동산'])

# 엑셀 저장 경로 설정
path = '/content/drive/MyDrive/python/crawling/hillstate_0212_2029v.xlsx'
df.to_excel(path, index=False)

print(f"데이터가 '{path}' 경로에 엑셀 파일로 저장되었습니다!")


동호수: ['우장산힐스테이트 130동', '우장산힐스테이트 120동', '우장산힐스테이트 120동', '우장산힐스테이트 138동', '우장산힐스테이트 138동', '우장산힐스테이트 138동', '우장산힐스테이트 138동', '우장산힐스테이트 106동', '우장산힐스테이트 130동', '우장산힐스테이트 101동', '우장산힐스테이트 106동', '우장산힐스테이트 133동', '우장산힐스테이트 123동', '우장산힐스테이트 129동', '우장산힐스테이트 123동', '우장산힐스테이트 116동', '우장산힐스테이트 127동', '우장산힐스테이트 103동', '우장산힐스테이트 134동', '우장산힐스테이트 130동']
거래 방식: ['전세', '매매', '매매', '전세', '전세', '전세', '전세', '매매', '매매', '매매', '매매', '매매', '매매', '매매', '매매', '매매', '전세', '매매', '매매', '매매']
가격: ['6억', '16억', '14억 7,000', '7억 5,000', '7억 5,000', '7억 5,000', '7억 5,000', '12억 3,000', '11억 1,000', '12억 8,000', '12억 3,000', '14억 5,000', '16억 5,000', '18억', '16억 5,000', '15억 9,000', '10억', '11억 5,000', '13억 5,000', '11억 1,000']
물건설명: ['79/59m², 14/18층, 남향 / 남향, 3월19일이후 하시 입주가능, 반전세가능', '132/101m², 14/17층, 동향 / 발산역 남향  확장 및 깨끗 투자가능물건', '132/101m², 1/17층, 남향 / 남향 거실, 방1개 확장, 매우 깨끗,  올수리, 입주 협의', '112/84m², 19/20층, 남서향 / 남향로얄층이며 깨끗한집 2월27일이후하시  빠른입주가능합니다', '112/84m², 19/20층, 남서향 / 올수리.환하고 예쁜집.빠른입주가', '112/84m², 19/2

In [ ]:
!pip install selenium

##방법1

In [ ]:
#전처리: Deepseek로 수정한 코드

import pandas as pd
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def get_data(article_no):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL&ad=true&articleNo='
    url = f"{base_url}{article_no}"

    driver.get(url)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    phone_numbers = [dd.get_text() for dd in soup.find_all('dd', class_='text text--number')]
    real_estate_offices = [strong.get_text() for strong in soup.find_all('strong', class_='info_title')]

    apartment_info = []
    for h4 in soup.find_all('h4', class_='info_title'):
        title_name = h4.find('strong', class_='info_title_name').get_text() if h4.find('strong', class_='info_title_name') else ''
        floor_info = h4.get_text(strip=True).replace(title_name, '')
        apartment_info.append(f"{title_name} {floor_info}")

    driver.quit()

    return phone_numbers, real_estate_offices, apartment_info

def extract_price(apartment_str):
    """ '매매' 키워드 오른쪽 문자열을 숫자형으로 변환 """
    match = re.search(r'매매\s*([\d억천만, ]+)', apartment_str)
    if match:
        price_str = match.group(1)
        return convert_price_to_int(price_str)
    return None

def convert_price_to_int(price_str):
    price_str = price_str.replace(',', '').replace(' ', '').strip()
    total_price = 0

    # '억' 단위 처리
    if '억' in price_str:
        parts = re.split(r'억', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**8
        price_str = parts[1] if len(parts) > 1 else ''

    # '천' 단위 처리 (천만원)
    if '천' in price_str:
        parts = re.split(r'천', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**7
        price_str = parts[1] if len(parts) > 1 else ''

    # '만' 단위 처리
    if '만' in price_str:
        parts = re.split(r'만', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**4
        price_str = parts[1] if len(parts) > 1 else ''

    # 남은 숫자 처리 (만원 단위로 가정)
    if price_str and price_str.isdigit():
        total_price += int(price_str) * 10**4

    return total_price if total_price != 0 else None

article_numbers = list(set([
    2504822211, 2504210136, 2505908173, 2505703456, 2503752598, 2501572048, 2502735210, 2502294399,
    2501662043, 2502293027, 2502356205, 2502814467, 2505703905, 2501607928, 2501658024, 2505050574,
    2501996356, 2504771773, 2502179793, 2504023203, 2503791928, 2502618284, 2505796375, 2503086595,
    2504444743, 2503516163, 2504694825, 2502357964, 2502025913, 2502353435, 2503170922, 2506080772
]))

data_list = []
serial_number = 1

for article_no in article_numbers:
    phone_numbers, real_estate_offices, apartment_info = get_data(article_no)
    apartment_str = ' '.join(apartment_info)  # 아파트 정보 통합
    price = extract_price(apartment_str)

    # 중복된 매물번호 체크
    existing_articles = [row[1] for row in data_list]
    if article_no not in existing_articles:
        data_list.append([
            serial_number,
            article_no,
            apartment_str,
            price,
            real_estate_offices,
            phone_numbers
        ])
        serial_number += 1

# 엑셀 저장 경로 설정
path = '/content/drive/MyDrive/python/crawling/hillstate.xlsx'
df = pd.DataFrame(data_list, columns=['일련번호', '매물번호', '아파트 동 호수', '가격', '부동산', '전화번호'])

# 엑셀 파일 저장
df.to_excel(path, index=False)

# 결과 출력
print(f"데이터가 '{path}' 경로에 엑셀 파일로 저장되었습니다!")
print("\n===== 수집된 데이터 =====\n")
print(df.head(10))

데이터가 '/content/drive/MyDrive/python/crawling/hillstate.xlsx' 경로에 엑셀 파일로 저장되었습니다!

===== 수집된 데이터 =====

   일련번호        매물번호                                        아파트 동 호수  \
0     1  2502814467  우장산힐스테이트 115동 13층매매13억 7,000 우장산힐스테이트 115동 13층   
1     2  2503086595          우장산힐스테이트 140동 고층매매15억 우장산힐스테이트 140동 고층   
2     3  2503516163        우장산힐스테이트 115동 16층매매15억 우장산힐스테이트 115동 16층   
3     4  2506080772    우장산힐스테이트 134동 3층매매13억 5,000 우장산힐스테이트 134동 3층   
4     5  2503752598    우장산힐스테이트 117동 중층매매13억 4,000 우장산힐스테이트 117동 중층   
5     6  2505796375          우장산힐스테이트 140동 고층매매15억 우장산힐스테이트 140동 고층   
6     7  2502353435        우장산힐스테이트 102동 12층매매15억 우장산힐스테이트 102동 12층   
7     8  2505703456    우장산힐스테이트 117동 중층매매13억 4,000 우장산힐스테이트 117동 중층   
8     9  2502293027    우장산힐스테이트 134동 저층매매13억 5,000 우장산힐스테이트 134동 저층   
9    10  2504023203                                                   

             가격                   부동산  \
0  1.370000e+09       [샘터부동산공인중개사사무소]   
1  1.500000e+09       [부동산뉴스공인중개사

##방법2

In [ ]:
#처음에 chatGPT legacy모델로 만든 코드

from selenium import webdriver
from bs4 import BeautifulSoup
import time

def get_data(article_no):
    # 크롬 드라이버 설정 (드라이버 경로 수정 필요)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 브라우저 창 없이 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    # 유동적인 URL 설정
    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL&ad=true&articleNo='

    url = f"{base_url}{article_no}"

    driver.get(url)

    # 페이지 로딩 대기
    time.sleep(5)

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 전화번호 크롤링
    phone_numbers = [dd.get_text() for dd in soup.find_all('dd', class_='text text--number')]

    # 부동산 사무소 이름 크롤링
    real_estate_offices = [strong.get_text() for strong in soup.find_all('strong', class_='info_title')]

    # 아파트 동 및 호수 정보 추출
    apartment_info = []
    for h4 in soup.find_all('h4', class_='info_title'):
        title_name = h4.find('strong', class_='info_title_name').get_text() if h4.find('strong', class_='info_title_name') else ''
        floor_info = h4.get_text(strip=True).replace(title_name, '')  # 동 정보 제거 후 층 정보만 남기기
        apartment_info.append(f"{title_name} {floor_info}")

    driver.quit()

    return phone_numbers, real_estate_offices, apartment_info


# 주어진 article 번호 리스트
article_numbers = [
    2504822211, 2504210136, 2505908173, 2505703456, 2503752598, 2501572048, 2502735210, 2502294399,
    2501662043, 2502293027, 2502356205, 2502814467, 2505703905, 2501607928, 2501658024, 2505050574,
    2501996356, 2504771773, 2502179793, 2504023203, 2503791928, 2502618284, 2505796375, 2503086595,
    2504444743, 2503516163, 2504694825, 2502357964, 2502025913, 2502353435, 2503170922
]

# 결과 출력
for article_no in article_numbers:
    phone_numbers, real_estate_offices, apartment_info = get_data(article_no)
    print(f"매물번호: {article_no}")
    print("아파트 동 호수:", apartment_info if apartment_info else "정보 없음")
    print("전화번호:", phone_numbers)
    print("부동산:", real_estate_offices)
    print('-' * 50)

#방법3
##직접 '전체거래방식'button 클릭하고 '전체면적'버튼 클릭하고 매물번호 추출하는 코드만들기 2025.02.08

In [ ]:
#전처리2: 직접 '전체거래방식'button 클릭하고 '전체면적'버튼 클릭하고 매물번호 추출하는 코드만들기 2025.02.08

import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time


def get_data(article_no):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL&ad=true&articleNo='
    url = f"{base_url}{article_no}"

    driver.get(url)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    #전체거래방식 클릭
    transaction = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[1]/button/span')
    transaction.click()


    #매매 버튼 클릭
    purchase = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[1]/div/div/ul/li[2]/label')
    purchase.click()


    #면적 클릭
    area = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[2]/button/span')
    area.click()

    #109m
    number_109 = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[2]/div/div/ul/li[3]/label')
    number_109.click()


    #112m 클릭
    phone_numbers = [dd.get_text() for dd in soup.find_all('dd', class_='text text--number')]
    real_estate_offices = [strong.get_text() for strong in soup.find_all('strong', class_='info_title')]

    apartment_info = []
    for h4 in soup.find_all('h4', class_='info_title'):
        title_name = h4.find('strong', class_='info_title_name').get_text() if h4.find('strong', class_='info_title_name') else ''
        floor_info = h4.get_text(strip=True).replace(title_name, '')
        apartment_info.append(f"{title_name} {floor_info}")

    driver.quit()

    return phone_numbers, real_estate_offices, apartment_info

def extract_price(apartment_str):
    """ '매매' 키워드 오른쪽 문자열을 숫자형으로 변환 """
    match = re.search(r'매매\s*([\d억천만, ]+)', apartment_str)
    if match:
        price_str = match.group(1)
        return convert_price_to_int(price_str)
    return None

def convert_price_to_int(price_str):
    price_str = price_str.replace(',', '').replace(' ', '').strip()
    total_price = 0

    # '억' 단위 처리
    if '억' in price_str:
        parts = re.split(r'억', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**8
        price_str = parts[1] if len(parts) > 1 else ''

    # '천' 단위 처리 (천만원)
    if '천' in price_str:
        parts = re.split(r'천', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**7
        price_str = parts[1] if len(parts) > 1 else ''

    # '만' 단위 처리
    if '만' in price_str:
        parts = re.split(r'만', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**4
        price_str = parts[1] if len(parts) > 1 else ''

    # 남은 숫자 처리 (만원 단위로 가정)
    if price_str and price_str.isdigit():
        total_price += int(price_str) * 10**4

    return total_price if total_price != 0 else None

article_numbers = list(set([
    2504822211, 2504210136, 2505908173, 2505703456, 2503752598, 2501572048, 2502735210, 2502294399,
    2501662043, 2502293027, 2502356205, 2502814467, 2505703905, 2501607928, 2501658024, 2505050574,
    2501996356, 2504771773, 2502179793, 2504023203, 2503791928, 2502618284, 2505796375, 2503086595,
    2504444743, 2503516163, 2504694825, 2502357964, 2502025913, 2502353435, 2503170922, 2506080772
]))

data_list = []
serial_number = 1

for article_no in article_numbers:
    phone_numbers, real_estate_offices, apartment_info = get_data(article_no)
    apartment_str = ' '.join(apartment_info)  # 아파트 정보 통합
    price = extract_price(apartment_str)

    # 중복된 매물번호 체크
    existing_articles = [row[1] for row in data_list]
    if article_no not in existing_articles:
        data_list.append([
            serial_number,
            article_no,
            apartment_str,
            price,
            real_estate_offices,
            phone_numbers
        ])
        serial_number += 1

# 엑셀 저장 경로 설정
path = '/content/drive/MyDrive/python/crawling/hillstate.xlsx'
df = pd.DataFrame(data_list, columns=['일련번호', '매물번호', '아파트 동 호수', '가격', '부동산', '전화번호'])

# 엑셀 파일 저장
df.to_excel(path, index=False)

# 결과 출력
print(f"데이터가 '{path}' 경로에 엑셀 파일로 저장되었습니다!")
print("\n===== 수집된 데이터 =====\n")
print(df.head(10))

In [ ]:

#전체거래방식 클릭
transaction = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[1]/button/span')
transaction.click()


#매매 버튼 클릭
purchase = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[1]/div/div/ul/li[2]/label')
purchase.click()


#전체면적 클릭
area = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[2]/button/span')
area.click()


#109m
number_109 = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[2]/div/div/ul/li[3]/label')
number_109.click()


#112m 클릭
number_112 = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[2]/div/div/ul/li[4]/label')
number_112.click()


#면적 X클릭
changex = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[2]/div/button')
changex.click()

#가격순
pricewise = driver.find_element(By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[3]/a[3]')
pricewise.click()

#방법4: 첫번째 페이지만 가져오기

#동적 스크롤 화면

In [23]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def scroll_down(driver):
    """동적 스크롤을 끝까지 수행"""
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # AJAX 데이터 로드 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def get_data():
    # Chrome WebDriver 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    # URL 접속
    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL'
    driver.get(base_url)
    time.sleep(5)  # 페이지 초기 로드 대기

    # 스크롤 다운
    scroll_down(driver)
    time.sleep(3)

    # 매물 정보 수집
    items = driver.find_elements(By.CSS_SELECTOR, 'div.item_list--article div.item_link')
    print(f"총 {len(items)}개의 매물 발견")

    apartment_info, transaction_types, prices, property_specs, real_estate_offices = [], [], [], [], []

    for index, item in enumerate(items, 1):
        try:
            apt_name = item.find_element(By.CSS_SELECTOR, 'span.text').text
            t_type = item.find_element(By.CSS_SELECTOR, 'span.type').text
            price = item.find_element(By.CSS_SELECTOR, 'span.price').text

            # 물건 설명
            spec_elements = item.find_elements(By.CSS_SELECTOR, 'span.spec')
            spec_text = ', '.join([spec.text for spec in spec_elements])

            # 부동산 정보
            try:
                agent_name = item.find_element(By.CSS_SELECTOR, 'span.agent_name').text
            except:
                agent_name = '정보 없음'

            # 데이터 리스트에 추가
            apartment_info.append(apt_name)
            transaction_types.append(t_type)
            prices.append(price)
            property_specs.append(spec_text)
            real_estate_offices.append(agent_name)

            print(f"{index}/{len(items)} 처리 완료: {apt_name}, 가격: {price}")

        except Exception as e:
            print(f"⚠️ {index}번 항목 처리 중 오류: {str(e)}")
            continue

    driver.quit()
    return apartment_info, transaction_types, prices, property_specs, real_estate_offices

# 데이터 수집 실행
apartment_info, transaction_types, prices, property_specs, real_estate_offices = get_data()

# 수집된 데이터 출력
print("동호수:", apartment_info)
print("거래 방식:", transaction_types)
print("가격:", prices)
print("물건설명:", property_specs)
print("부동산:", real_estate_offices)

# 데이터프레임으로 저장
data_list = list(zip(apartment_info, transaction_types, prices, property_specs, real_estate_offices))
df = pd.DataFrame(data_list, columns=['동호수', '거래 방식', '가격', '물건 설명', '부동산'])

# 엑셀 저장 경로 설정
#path = 'hillstate_new.xlsx'
path = '/content/drive/MyDrive/python/crawling/hillstate_1722.xlsx'

df.to_excel(path, index=False)

print(f"\n✅ 데이터가 '{path}' 경로에 엑셀 파일로 저장되었습니다!")


총 0개의 매물 발견
동호수: []
거래 방식: []
가격: []
물건설명: []
부동산: []

✅ 데이터가 '/content/drive/MyDrive/python/crawling/hillstate_1722.xlsx' 경로에 엑셀 파일로 저장되었습니다!


#방법5: scroll down()함수를 만들어 전체 매물 검색시도
##우장산힐스테이트 전체 매물 추출

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def scroll_down(driver):
    """점진적 스크롤 방식 적용 및 새로운 요소 로드 확인"""
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 명시적 대기 (새 요소 로딩 기다림)
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".item_link"))
            )
        except Exception as e:
            print("로드 대기 중 오류:", e)

        time.sleep(2)  # AJAX 요청 대기

        # 스크롤 완료 검증
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


def get_data():
    # Chrome 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)

    # URL 접속
    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL'
    driver.get(base_url)
    time.sleep(5)

    # 스크롤 다운 실행
    scroll_down(driver)
    time.sleep(3)

    # 데이터 수집
    items = driver.find_elements(By.CSS_SELECTOR, '.item_link')
    print(f'▶ 발견된 매물 수: {len(items)}개')

    # 데이터 저장 리스트
    apartment_info, transaction_types, prices, property_specs, real_estate_offices = [], [], [], [], []

    for index, item in enumerate(items, 1):
        try:
            apt = item.find_element(By.CSS_SELECTOR, '.text').text
            t_type = item.find_element(By.CSS_SELECTOR, '.type').text
            price = item.find_element(By.CSS_SELECTOR, '.price').text

            specs = item.find_elements(By.CSS_SELECTOR, '.spec')
            spec_text = ', '.join([s.text for s in specs])

            try:
                estate = item.find_element(By.CSS_SELECTOR, '.agent_name').text
            except:
                estate = '정보 없음'

            apartment_info.append(apt)
            transaction_types.append(t_type)
            prices.append(price)
            property_specs.append(spec_text)
            real_estate_offices.append(estate)

            print(f'{index}/{len(items)} 처리 완료: {apt}')

        except Exception as e:
            print(f'⚠️ {index}번 항목 처리 중 오류: {str(e)}')
            continue

    driver.quit()
    return apartment_info, transaction_types, prices, property_specs, real_estate_offices


if __name__ == "__main__":
    apartment_info, transaction_types, prices, property_specs, real_estate_offices = get_data()

    data = {
        '동호수': apartment_info,
        '거래 방식': transaction_types,
        '가격': prices,
        '물건 설명': property_specs,
        '부동산': real_estate_offices
    }
    df = pd.DataFrame(data)

    print("\n■ 수집 결과 요약 ■")
    print(f"총 {len(df)}개의 매물 정보 수집 완료")
    print(df.head(3))

    #save_path = 'hillstate_1442.xlsx'
    save_path = '/content/drive/MyDrive/python/crawling/hillstate_1453.xlsx'
    df.to_excel(save_path, index=False)
    df.to_excel(save_path, index=False)
    print(f"\n✅ 데이터가 성공적으로 저장되었습니다: {save_path}")

▶ 발견된 매물 수: 20개
1/20 처리 완료: 우장산힐스테이트 134동
2/20 처리 완료: 우장산힐스테이트 130동
3/20 처리 완료: 우장산힐스테이트 138동
4/20 처리 완료: 우장산힐스테이트 120동
5/20 처리 완료: 우장산힐스테이트 120동
6/20 처리 완료: 우장산힐스테이트 120동
7/20 처리 완료: 우장산힐스테이트 103동
8/20 처리 완료: 우장산힐스테이트 106동
9/20 처리 완료: 우장산힐스테이트 101동
10/20 처리 완료: 우장산힐스테이트 101동
11/20 처리 완료: 우장산힐스테이트 130동
12/20 처리 완료: 우장산힐스테이트 106동
13/20 처리 완료: 우장산힐스테이트 103동
14/20 처리 완료: 우장산힐스테이트 106동
15/20 처리 완료: 우장산힐스테이트 116동
16/20 처리 완료: 우장산힐스테이트 113동
17/20 처리 완료: 우장산힐스테이트 113동
18/20 처리 완료: 우장산힐스테이트 120동
19/20 처리 완료: 우장산힐스테이트 106동
20/20 처리 완료: 우장산힐스테이트 120동

■ 수집 결과 요약 ■
총 20개의 매물 정보 수집 완료
             동호수 거래 방식         가격  \
0  우장산힐스테이트 134동    매매        14억   
1  우장산힐스테이트 130동    매매  11억 1,000   
2  우장산힐스테이트 138동    전세   7억 5,000   

                                               물건 설명    부동산  
0            112/84m², 7/21층, 남서향, 주인거주,입주가, 깨끗한 남향집  정보 없음  
1  79/59m², 14/18층, 남향, 판상형 남향 로얄층 방1개확장 광폭베란다 우장...  정보 없음  
2    112/84m², 19/20층, 남서향, 남서,뷰굿,기본,2월말,3월중순가능,입주협의  정보 없음  

✅ 데이터가 성공적으로 저장되었습니다: 

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def scroll_down(driver):
    """점진적 스크롤 방식 적용 및 새로운 요소 로드 확인"""
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # AJAX 요청 대기

        # 새로운 요소 로드 확인
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            try:
                # 모든 요소가 로드되었는지 확인
                WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".item_link"))
                )
            except Exception as e:
                print("모든 요소 로드 완료 또는 타임아웃:", e)
            break
        last_height = new_height


def get_data():
    # Chrome 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)

    # URL 접속
    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL'
    driver.get(base_url)
    time.sleep(5)

    # 스크롤 다운 실행
    scroll_down(driver)
    time.sleep(3)

    # 페이지 소스 확인을 위한 스냅샷 저장
    page_source = driver.page_source
    with open("page_snapshot.html", "w", encoding="utf-8") as f:
        f.write(page_source)
    print("페이지 스냅샷 저장 완료")

    # 데이터 수집
    items = driver.find_elements(By.CSS_SELECTOR, '.item_link')
    print(f'▶ 발견된 매물 수: {len(items)}개')

    # 데이터 저장 리스트
    apartment_info, transaction_types, prices, property_specs, real_estate_offices = [], [], [], [], []

    for index, item in enumerate(items, 1):
        try:
            apt = item.find_element(By.CSS_SELECTOR, '.text').text
            t_type = item.find_element(By.CSS_SELECTOR, '.type').text
            price = item.find_element(By.CSS_SELECTOR, '.price').text

            specs = item.find_elements(By.CSS_SELECTOR, '.spec')
            spec_text = ', '.join([s.text for s in specs])

            try:
                estate = item.find_element(By.CSS_SELECTOR, '.agent_name').text
            except:
                estate = '정보 없음'

            apartment_info.append(apt)
            transaction_types.append(t_type)
            prices.append(price)
            property_specs.append(spec_text)
            real_estate_offices.append(estate)

            print(f'{index}/{len(items)} 처리 완료: {apt}')

        except Exception as e:
            print(f'⚠️ {index}번 항목 처리 중 오류: {str(e)}')
            continue

    driver.quit()
    return apartment_info, transaction_types, prices, property_specs, real_estate_offices


if __name__ == "__main__":
    apartment_info, transaction_types, prices, property_specs, real_estate_offices = get_data()

    data = {
        '동호수': apartment_info,
        '거래 방식': transaction_types,
        '가격': prices,
        '물건 설명': property_specs,
        '부동산': real_estate_offices
    }
    df = pd.DataFrame(data)

    print("\n■ 수집 결과 요약 ■")
    print(f"총 {len(df)}개의 매물 정보 수집 완료")
    print(df.head(3))

    # 데이터 저장 경로 설정
    save_path = '/content/drive/MyDrive/python/crawling/hillstate_1715.xlsx'

    df.to_excel(save_path, index=False)
    print(f"\n✅ 데이터가 성공적으로 저장되었습니다: {save_path}")


페이지 스냅샷 저장 완료
▶ 발견된 매물 수: 20개
1/20 처리 완료: 우장산힐스테이트 134동
2/20 처리 완료: 우장산힐스테이트 120동
3/20 처리 완료: 우장산힐스테이트 130동
4/20 처리 완료: 우장산힐스테이트 138동
5/20 처리 완료: 우장산힐스테이트 103동
6/20 처리 완료: 우장산힐스테이트 101동
7/20 처리 완료: 우장산힐스테이트 106동
8/20 처리 완료: 우장산힐스테이트 120동
9/20 처리 완료: 우장산힐스테이트 101동
10/20 처리 완료: 우장산힐스테이트 106동
11/20 처리 완료: 우장산힐스테이트 103동
12/20 처리 완료: 우장산힐스테이트 130동
13/20 처리 완료: 우장산힐스테이트 120동
14/20 처리 완료: 우장산힐스테이트 120동
15/20 처리 완료: 우장산힐스테이트 120동
16/20 처리 완료: 우장산힐스테이트 113동
17/20 처리 완료: 우장산힐스테이트 106동
18/20 처리 완료: 우장산힐스테이트 116동
19/20 처리 완료: 우장산힐스테이트 113동
20/20 처리 완료: 우장산힐스테이트 106동

■ 수집 결과 요약 ■
총 20개의 매물 정보 수집 완료
             동호수 거래 방식         가격  \
0  우장산힐스테이트 134동    매매        14억   
1  우장산힐스테이트 120동    매매  15억 5,000   
2  우장산힐스테이트 130동    매매  11억 1,000   

                                               물건 설명    부동산  
0            112/84m², 7/21층, 남서향, 주인거주,입주가, 깨끗한 남향집  정보 없음  
1  132/101m², 2/17층, 남향, 남향 로얄동 올확장 깨끗 입주가능 주인거주 ...  정보 없음  
2  79/59m², 14/18층, 남향, 판상형 남향 로얄층 방1개확장 광폭베란다 우장...  정보 없음  

✅ 데이터가 성

In [ ]:
#전처리한 파일 불러오기
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

#matplotlib에서 한글구현
plt.rc('font', family='NanumBarunGothic')

#그래프 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

path = '/content/drive/MyDrive/python/crawling/hillstate.xlsx'
df = pd.read_excel(path)
df.head()

,일련번호,매물번호,아파트 동 호수,가격,부동산,전화번호
0,1,2502814467,"우장산힐스테이트 115동 13층매매13억 7,000 우장산힐스테이트 115동 13층",1.370000e+09,['샘터부동산공인중개사사무소'],"['서울 강서구 내발산동657 우장산힐스테이트 상가1동 102-3', '02-265..."
1,2,2503086595,우장산힐스테이트 140동 고층매매15억 우장산힐스테이트 140동 고층,1.500000e+09,['부동산뉴스공인중개사사무소'],"['서울 강서구 등촌동694 대림아파트 상가105호', '02-3664-4900,0..."
2,3,2503516163,우장산힐스테이트 115동 16층매매15억 우장산힐스테이트 115동 16층,1.500000e+09,['(주)우대빵부동산중개법인 강서지점'],"['서울 강서구 화곡동1006-9 2층', '1661-9539,010-2805-58..."
3,4,2506080772,"우장산힐스테이트 134동 3층매매13억 5,000 우장산힐스테이트 134동 3층",1.350000e+09,['명문효자부동산공인중개사사무소'],"['서울특별시 강서구 강서로 348 1층(내발산동.우장산힐스테이트상가)', '02-..."
4,5,2503752598,"우장산힐스테이트 117동 중층매매13억 4,000 우장산힐스테이트 117동 중층",1.340000e+09,['발산공인중개사사무소'],"['서울특별시 강서구 내발산동 688 1층', '02-2665-8984,010-9..."
